In [ ]:
! pip install torch

## Téléchargement et format des données

Nous avons un fichier a 7 colonnes

MOTS ---- POS ---- HEAD_ID ---- DEPREL ---- TARGET_OR_NOT ---- LABEL_ENCODE_1 ---- LABEL_ENCODE_2

---
Les 4 premières vous les connaissez déjà, elles ont été predites automatiquement avec un analyzeur syntaxique (UDPipe)

Les 3 dernières sont des colonnes associées à la semantique en frames (FrameNet)

**TARGET_OR_NOT**: indique si le mot peut déclancher une frame (T) ou pas (N)

**LABEL_ENCODE_1**: une façon de encoder les labels semantiques     [BIO] + [FE ou TG] + [ROLE or FRAME]

**LABEL_ENCODE_2**: une deuxième façon de encoder les labels semantiques     [BIO] + FRAME_NAME + [FE ou TG] + [ROLE or LEXICAL_UNIT]


---

Dans ce fichier il y a déjà eu un pretraitement pour selectioner quels sont les TARGET possibles. Si une phrase a 2 TARGETS ou plus elle a été dupliqué autant de fois qu'il y avait de TARGETS, pour pouvoir traiter un TARGET à la fois.



In [ ]:
%%bash
if [ ! -f frames.data ]; then
    wget -q https://pageperso.lis-lab.fr/gabriel.marzinotto/frames.data
fi
head -20 frames.data

La première chose à faire est charger les données. Un premier tableau contiendra les mots de chaque phrase, et un second les POS, puis un pour TARGET_OR_NOT et pous de tableaux pour les étiquettes encodées.

In [ ]:
list_text = []
list_pos = []
list_tg_or_not = []
list_labels_1 = []
list_labels_2 = []

with open('frames.data') as fp:
    text = []
    pos = []
    tg_tag = []
    label_1 = []
    label_2 = []
    
    for line in fp:
        tokens = line.strip().split('\t')
        #print(tokens)
        if len(tokens) > 1:
            text.append(tokens[0])
            pos.append(tokens[1])
            tg_tag.append(tokens[4])
            label_1.append(tokens[5])
            label_2.append(tokens[6])

        else:
            list_text.append(text)
            list_pos.append(pos)
            list_tg_or_not.append(tg_tag)
            list_labels_1.append(label_1)
            list_labels_2.append(label_2)

            text = []
            pos = []
            tg_tag = []
            label_1 = []
            label_2 = []

print(len(list_text))

print(list_text[12])
print(list_labels_1[12])

Il faut ensuite convertir les étiquettes et les mots en entiers. Nous allons devoir créer des séquences de taille fixe, donc il faut réserver une étiquette de padding, ici `<eos>`. Toutefois, pour l'instant nous nous contentons de convertir les étiquettes sans appliquer le padding. Un defaultdict est utilisé pour créer le vocabulaire des étiquettes. À chaque fois qu'une nouvelle étiquette est rencontrée, il l'associe à une nouvelle valeur.

In [ ]:
import collections
label_vocab = collections.defaultdict(lambda: len(label_vocab))
label_vocab['<eos>'] = 0

int_labels = []
for label in list_labels_1:
    int_labels.append([label_vocab[token] for token in label])

print(int_labels[12])
print(label_vocab)

Il en va de même pour les textes à qui nous allons dédier un vocabulaire différent de celui des étiquettes. Les embeddings préentraînés seront chargés à partir de ce vocabulaire. Il faut veiller à mettre les mots en minuscules car le vocabulaire des embeddings fasttext est en minuscules.

In [ ]:
vocab = collections.defaultdict(lambda: len(vocab))
vocab['<eos>'] = 0

int_texts = []
for text in list_text:
    int_texts.append([vocab[token.lower()] for token in text])

print(int_texts[12])
len(vocab)

Egalement nous avons besoin de un input pour indiquer quel est la TARGET. Autrement, s'il y a plusieurs targets dans une phrase, le modèle ne pourrais pas faire la distinction.

In [ ]:
tg_indicator_vocab = collections.defaultdict(lambda: len(tg_indicator_vocab))
tg_indicator_vocab['<eos>'] = 0

int_tg_indicator = []
for text in list_tg_or_not:
    int_tg_indicator.append([tg_indicator_vocab[token] for token in text])

print(int_tg_indicator[12])
len(tg_indicator_vocab)

Nous pouvons maintenant créer des vocabulaires inversés pour pouvoir revenir vers les étiquettes et mots. Celà et utile pour vérifier le contenu d'un tenseur, et surtout pour convertir les décisions du système en étiquettes textuelles.

In [ ]:
rev_label_vocab = {y: x for x, y in label_vocab.items()}
rev_vocab = {y: x for x, y in vocab.items()}

Nous pouvons maintenant calculer la longueur moyenne de nos exemples de apprentissage.
Ceci nous permettra définir une max_length pertinante pour apprendre notre resseaux
(Pas trop de padding)

In [ ]:
! pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

lengths = []
len_texts = [ len(text) for text in list_text ]

plt.hist(len_texts)

Importons les modules habituels de pytorch. Les constantes suivantes sont définies :

    max_len est la longueur maximale d'une phrase en apprentissage
    batch_size est la taille des batches
    embed_size est la taille des embeddings préentraînés (nous utiliserons les embeddings téléchargeables sur le site de fasttext qui sont de taille 300.
    hidden_size est la taille de l'état caché du RNN



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

max_len = 50
batch_size = 64
embed_size = 300
hidden_size = 128

Il faut maintenant créer des tenseurs pytorch avec les données phrases et les étiquettes associées. X et Y sont des tenseurs d'entiers de taille (le nombre d'exemples du corpus, la longueur maximale d'une phrase). Ils sont initialisés à zéro car c'est la valeur de padding.

Pour chaque phrase et séquence d'étiquettes associées, nous calculons la longueur effective à intégrer en fonction de la longueur maximale (les phrases et séquences d'étiquettes trop longues sont coupées). Puis nous pouvons les intégrer dans les tenseurs en début de ligne, les fins de lignes étant remplies de zéros (`<eos>`).


In [ ]:
X = torch.zeros(len(int_texts), max_len).long()
T = torch.zeros(len(int_texts), max_len).long()
Y = torch.zeros(len(int_labels), max_len).long()

for i, (text, tg, label) in enumerate(zip(int_texts, int_tg_indicator, int_labels)):
    length = min(max_len, len(text))
    X[i,:length] = torch.LongTensor(text[:length])
    T[i,:length] = torch.LongTensor(tg[:length])
    Y[i,:length] = torch.LongTensor(label[:length])

print(X[12])
print(T[12])
print(Y[12])

Pour vérifier les performances du système, nous le séparons en un ensemble d'entraînement et un ensemble de développement. Comme il y a 7605 exemple, nous utiliserons les 6000 premiers pour l'entraînement et les 1605 suivants pour la validation.



In [ ]:
X_train = X[:6000]
T_train = T[:6000]
Y_train = Y[:6000]

X_valid = X[6000:]
T_valid = T[6000:]
Y_valid = Y[6000:]

Torch contient des classes permettant facilement de charger des batches d'exemples déjà mélangés pour l'entraînement. Nous allons en tirer parti.


In [ ]:
from torch.utils.data import TensorDataset, DataLoader
train_set = TensorDataset(X_train, T_train, Y_train)
valid_set = TensorDataset(X_valid, T_valid, Y_valid)

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size)

La prochaine étape consiste en le chargement des embeddings préentraînés. Les embeddings fasttext peuvent être téléchargés depuis la page suivante : 

https://github.com/facebookresearch/fastText/blob/master/pretrained-vectors.md

Toutefois, ces embeddings couvrant plusieurs millions de mots, le fichier fait plusieurs GiB et une version filtrée par rapport au vocabulaire du corpus est disponible.


In [ ]:
![ -f wiki.fr.vec.small ] || wget -q http://pageperso.lif.univ-mrs.fr/gabriel.marzinotto/wiki.fr.vec.small 

Les embeddings ont le format suivant :

    chaque ligne décrit l'embedding d'un mot
    le mot est sur la première colone, suivi des valeurs du vecteur dans l'espace de 300 dimension.

Nous créons donc un tenseur à zéro, puis plaçons l'embedding de chaque mot du vocabulaire rencontré à la bonne ligne dans le tenseur. Les embeddings des mots non couverts dans ce fichier resteront à zéro.


In [ ]:
pretrained_weights = torch.zeros(len(vocab), 300)
with open('wiki.fr.vec.small') as fp:
    for line in fp:
        tokens = line.strip().split()
        if tokens[0] in vocab:
            try:
              pretrained_weights[vocab[tokens[0]]] = torch.FloatTensor([float(x) for x in tokens[1:]])
            except:
              continue
                
pretrained_weights[12][:10]

Le modèle est une class qui étend nn.Module. Son constructeur appelle le constructeur de la class mère, puis déclare les différentes couches : une couche d'embeddings, une couche de GRU et une couche de décision. Cette couche de décision sera appliquée à chaque position de la phrase pour prédire l'étiquette associée.

Le premier détail important que nous allons traiter est le problème du padding. Pour pouvoir prendre en entrée des batches de phrases de la même taille, nous avons complété ces dernières avec des 0. Le mot d'indice zéro a un embedding et le système peut donc l'utiliser pour apprendre des régularités des données. Si on utilise un RNN bidirectionnel, l'état caché après chaque padding peut aussi contribuer à la représentation créée. Ces deux problème font que le modèle aura un comportement différent si on change la taille des séquences traitées.

Une solution est de forcer l'état caché à rester à zéro en présence de padding. Pour celà il faut spécifier le padding_idx de la couche d'emebdding pour la représentation associée au padding soit toujours le vecteur nul. Ensuite, dans le RNN, l'état caché est calculé comme une transformation affine à partir de l'embedding en entrée et de l'état caché précédent. Comme l'état caché initial est à zéro, et que l'embedding du padding est à zéro, si on désactive le bias dans les calculs (en donnant l'option bias=False au GRU), cela va forcer l'état caché à rester à zéro tout au long du padding. Le modèle peut ainsi traiter des séquences de taille arbitraire, même quand il est bidirectionnel.

Le deuxième détail est l'initialisation des embeddings. La méthode diffère selon les versions de pytorch, mais la façon présentée ici marche dans la plupart des cas. Nous allons directement modifier le champ weight de la couche d'embeddings et remplacer ce paramètre par nos embeddings préentraînés. requires_grad=False permet de geler la couche d'embeddings et donc de ne pas les modifier lors de l'apprentissage. Cela permettra en prédiction d'utiliser des embeddings fasttext pour les mots que nous n'avons pas observés dans le corpus d'apprentissage, en espérant qu'un mot d'embedding proche s'y trouvait. Si on omet cette option, les embeddings sont fine-tunés pour maximiser les performances du système.


In [ ]:
class RNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.word_embed = nn.Embedding(len(vocab), embed_size, padding_idx=vocab['<eos>'])
        self.word_embed.weight = nn.Parameter(pretrained_weights, requires_grad=False)
        
        #Initialize the embeddings for the TARGET_OR_NOT using the identity matrix
        tg_ind = len(tg_indicator_vocab)
        self.tg_embed = nn.Embedding(tg_ind, tg_ind, padding_idx=tg_indicator_vocab['<eos>'])
        self.tg_embed.weight = nn.Parameter(torch.eye(tg_ind), requires_grad=False)
        
        self.rnn = nn.GRU(embed_size+tg_ind, hidden_size, bias=False, num_layers=1, dropout=0.3, bidirectional=False, batch_first=True)
        self.decision = nn.Linear(hidden_size * 1 * 1, len(label_vocab))
        
    def forward(self, x, t):
        word_embed = self.word_embed(x)
        tg_embed = self.tg_embed(t)
        
        embed = torch.cat((word_embed,tg_embed),-1)
        
        output, hidden = self.rnn(embed)
        return self.decision(output)

rnn_model = RNN()
rnn_model

In [ ]:
rnn_model(Variable(X[:2]), Variable(T[:2])).size()

La fonction qui calcule les performances d'un modèle doit accomoder la nouvelle forme des données.
En particulier, le critère d'entropie croisée n'accepte que des matrices à deux dimensions, donc il faut redimentionner les scores produits pour qu'ils ait la taille `(batch_size * sequence_length, num_labels)` et les références pour quelles aient la taille `(batch_size * sequence_length)`.

Ensuite, il faut modifier le max qui calcule les prédictions pour qu'il agisse sur la dernière dimension du tenseur `y_scores`.

Et finalement, pour calculer le score d'une séquence, nous devons ignorer le padding. Pour celà une matrice `mask` est crée qui contient 1 pour les éléments non nuls de la matrice contenant les étiquettes et 0 sinon.
On peut calculer le nombre de corrects ainsi que le numérateur de la fonction de performance en appliquant le masque.

Le loss, pour être comparable au loss de l'entraînement, n'est pas modifié.

In [ ]:
def perf(model, loader):
    criterion = nn.CrossEntropyLoss()
    model.eval()
    total_loss = correct = num_loss = num_perf = 0
    for x, t, y in loader:
        y_scores = model(Variable(x, volatile=True), Variable(t, volatile=True))
        loss = criterion( y_scores.view(y.size(0) * y.size(1), -1), 
                          Variable(y.view(y.size(0) * y.size(1)), volatile=True) )
        y_pred = torch.max(y_scores, 2)[1]
        mask = (y != 0)
        correct += torch.sum((y_pred.data == y) * mask)
        total_loss += loss.data
        num_loss += len(y)
        num_perf += torch.sum(mask)
        
    #print("df", correct, num_perf)
    return total_loss / num_loss, float(correct) / float(num_perf)

perf(rnn_model, valid_loader)

La fonction d'apprentissage est modifiée en deux endroits. Tout d'abord, pytorch refuse d'entraîner un modèle contenant des paramètres sans gradient. Nous devons donc filtrer la liste des paramètres passés à l'optimiseur pour qu'elle ne contienne pas les embeddings "gelés".

Ensuite, nous appliquons le même redimenssionnement des scores et des équeittes pour accomoder le critère d'apprentissage.


In [ ]:
def fit(model, epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda param: param.requires_grad, model.parameters()))
    for epoch in range(epochs):
        model.train()
        total_loss = num = 0
        for x, t, y in train_loader:
            optimizer.zero_grad()
            y_scores = model(Variable(x), Variable(t))
            loss = criterion(y_scores.view(y.size(0) * y.size(1), -1), Variable(y.view(y.size(0) * y.size(1)), volatile=True))
            loss.backward()
            optimizer.step()
            total_loss += loss.data
            num += len(y)
        print(epoch, total_loss / num, perf(model, valid_loader))

fit(rnn_model, 20)

Une fois le modèle entraîné, nous pouvons écrire une fonction qui génère les prédictions pour une phrase. Cette fonction devra :

    convertir les mots en entiers (nous n'avons pas prévu de symbole pour les mots inconnus donc ils seront remplacés par du padding)
    faire un batch de taille 1 sous la forme d'un tenseur de dimensions (1, longueur de la phrase)
    calculer les scores des étiquettes à chaque position
    calculer les prédictions correspondantes
    et convertir les entiers prédits en text grâce au dictionnaire inversé créé plus haut

Notez que cette fonction peut prendre en entrée des phrases plus longues que max_len et que le temps de calcul n'est pas gaspillé par le padding.


In [ ]:
# we will use rev_label_vocab to go from ids -> labels
def fscore(model, loader):
  
    def build_arg_matrix(y_tensor):
        refs = collections.defaultdict(lambda: collections.defaultdict(lambda: collections.defaultdict()) )
        nb_refs = 0.0
        
        batch_size, length = y_tensor.shape

        for i_ in range(batch_size):
          y_sent = y_tensor[i_,:]
          start_elem = None
          end_elem   = None
          label_elem = None

          print(i_,nb_refs, len(y_tensor))
          for j,y in enumerate(y_sent):
  
            label = rev_label_vocab[y]
            # if padding...
            if(y == '<eos>'): 
              if( start_elem and end_elem and label_elem):
                refs[i_][start_elem][end_elem] = label_elem
                nb_refs += 1

              start_elem = None
              end_elem   = None
              label_elem = None
            
            
            elif(label[0] == "B"):
              
              if( start_elem and end_elem and label_elem):
                refs[i_][start_elem][end_elem] = label_elem
                nb_refs += 1
                
              start_elem = j
              end_elem   = j
              label_elem = label[2:]
              
              
            elif(label[0] == "I"):
                if(label[2:] == label_elem):
                  end_elem = j
                else:
                  if( start_elem and end_elem and label_elem):
                    refs[i_][start_elem][end_elem] = label_elem   
                    nb_refs += 1
              
                  start_elem = j
                  end_elem   = j
                  label_elem = label[2:]              
                              
            elif(label[0] == "O"):              
              if( start_elem and end_elem and label_elem):
                refs[i_][start_elem][end_elem] = label_elem
                nb_refs += 1

              start_elem = None
              end_elem   = None
              label_elem = None
          
          
          if( start_elem and end_elem and label_elem):
            refs[i_][start_elem][end_elem] = label_elem
            nb_refs += 1

        return refs, nb_refs
  
  
    def compare(y_true, y_pred):
        refs, nb_refs = build_arg_matrix(y_true.numpy())
        hyps, nb_hyps = build_arg_matrix(y_pred.numpy())
        
        #print(refs)
        #print('-'*50)
        #print(hyps)
        
        nb_ok   = 0.0
        counter = 0
        for i, dico in refs.items():
          for j, subdico in dico.items():
            for k, value in subdico.items():   
              if( i in hyps and
                  j in hyps[i] and 
                  k in hyps[i][j] and
                  hyps[i][j][k] == value ):
                nb_ok += 1.0   
              
        return nb_ok, nb_refs, nb_hyps

    nb_ok = nb_ref = nb_hyp = 0
    model.eval()
    
    for x, t, y in loader:
        y_scores = model(Variable(x, volatile=True), Variable(t, volatile=True))        
        y_pred = torch.max(y_scores, 2)[1]

        k, r, h = compare(y, y_pred)
        nb_ok  += k
        nb_ref += r
        nb_hyp += h

    print(nb_ok)
    print(nb_ref)
    print(nb_hyp)
    
    prec = nb_ok/nb_hyp
    recall = nb_ok/nb_ref
    f1 = 2*(prec*recall)/(prec+recall)
    
    return prec, recall, f1

fscore(rnn_model, valid_loader)